<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet
!pip install openai --quiet

host="localhost:11434"
modelid="chevalblanc/gpt-4o-mini"
modelid="deepseek-r1:14b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [2]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [3]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [4]:
from pydantic import Field, BaseModel

class Response(BaseModel):
   action: str = Field(description="the agent to select or \"respond_to_user\" when there are no agents to select from or there is no next_action")
   input: str = Field(description="the input for this agent")
   next_action: str = Field(description="the next agent to call after this agent")


In [5]:
def call(system_prompt: str, message: str, model: str = modelid, history: list[dict] = [] ) -> str:
  history.append(
          {
              "role": "user",
              "content": message,
          },)
  completion = llm.chat.completions.parse(
      model=modelid,
      messages=[ {"role": "system", "content": system_prompt},]
      +history,
      temperature=0.0,
      max_tokens=200,
      response_format=Response
  )

  return completion.choices[0].message.parsed

In [8]:
import json

def orchestrator(userinput: str, memory: list[str] = []) -> Response:

  character = "You are an intelligent agent orchestrator that coordinates between specialized agents to solve complex problems through multi-step planning and execution."
  prompt = """

                You are an intelligent agent orchestrator that coordinates between specialized agents to solve complex problems through multi-step planning and execution.

                Review chat history for context and previous interactions
                Analysise the user's input to classify the intent select the appropriate agent.
                Determine which agents are needed and in what order
                Call the appropriate agent with specific instructions
                Wait for the agent's response
                Evaluate the results

                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the current weather for a given city.

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the agent "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with agent "respond_to_user" when there are no agents to select from or there is no next_action


  """
#   print ("\n".join(memory))
  print (prompt.format(userinput=userinput))
  step = call (character, prompt.format(userinput=userinput),history=memory)

  for i in range (0,5):
    print (f"Orchestrator: {step}")
    memory.append({"role": "assistant", "content": step.model_dump_json()},)

    if step.action == "current_time_agent":
      response = "the current time in New York is 10:00AM in the morning"
    if step.action == "schedule_agent":
      return orchestrator(step.input)
    if step.action == "weather_agent":
      response = "The weather in New York is sunny with 39 degress Celsius"
    if step.action == "respond_to_user":
      return step.input
    step = call (character, response,history=memory)



In [9]:
result = orchestrator ("What is the current time and weather in New York ?")
result



                You are an intelligent agent orchestrator that coordinates between specialized agents to solve complex problems through multi-step planning and execution.

                Review chat history for context and previous interactions
                Analysise the user's input to classify the intent select the appropriate agent.
                Determine which agents are needed and in what order
                Call the appropriate agent with specific instructions
                Wait for the agent's response
                Evaluate the results

                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the curren

'The current time in New York is 10:00 AM and the weather is sunny with 39 degrees Celsius.'